In [ ]:
# Project Title: Custom Tabular LLM Chatbot 

In [ ]:
# Cell for importing required modules
import pandas as pd
from tabulate import tabulate

from langchain.text_splitter import MarkdownTextSplitter
from langchain.docstore.document import Document
import tiktoken
import openai

import time 
import os 
import re

In [ ]:
# Remove Pandas limitations to display entire output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Load the CSV file and verify the content
df = pd.read_csv("EPIQ.csv", index_col = None, header = 3)

# Get rid of NA values 
df = df.dropna(axis=1, how='all')

In [ ]:
df.info()

In [ ]:
# Extract metadata
metadata = {
    'columns': list(df.columns),
    'num_entries': len(df),
    'example_entry': df.iloc[0].to_dict()  
}

In [ ]:
# Convert to markdown
mkdn = tabulate(df, tablefmt="pipe", headers="keys")

In [ ]:
# Check total length of the input in tokens
def calculate_tokens(text, encoding):
    encoding = tiktoken.get_encoding(encoding)
    num_tokens = len(encoding.encode(text))
    return num_tokens

calculate_tokens(mkdn, "cl100k_base")

In [ ]:
def text_to_docs(text, metadata):
    doc_chunks = []
    text_splitter = MarkdownTextSplitter(chunk_size=250, chunk_overlap=0)  # only 1 row per doc, otherwise - parsing errors in prompting!!!
    chunks = text_splitter.split_text(text)
    for i, chunk in enumerate(chunks):
        doc = Document(page_content=chunk, metadata=metadata)
        doc_chunks.append(doc)
    return doc_chunks

In [ ]:
docs = text_to_docs(mkdn, metadata)

In [ ]:
del docs[0]    # removing the first doc which only contains headers 

In [ ]:
def get_next_file_num(directory, base_filename):
    # get a list of all existing filenames
    files = os.listdir(directory)
    
    # filter to only those that match the filename pattern
    base_filename = f'{base_filename}_'
    matching_files = [f for f in files if f.startswith(base_filename)]
    
    # extract the numbers from these filenames
    nums = [int(re.search(r'(\d+)', f).group()) for f in matching_files if re.search(r'(\d+)', f)]

    # return the max number + 1, or 1 if no existing files
    return max(nums) + 1 if nums else 1

In [ ]:
# Store conversation history in Excel for further reference 
def xlsx_log(file_name, user_message, bot_response):

    excel_log = pd.DataFrame({'User': [user_message], 'Bot': [bot_response]})
    num = get_next_file_num('.', file_name)
        
    excel_log.to_csv(f'{file_name}_{num}.csv', index=False)

In [ ]:
openai.api_key = "your_key"

In [ ]:
# Option alternative to relevance score - proved to be faster and more efficient since we are dealing with unique identifiers of similar structure
def extract_details(query):
    match = re.search(r"FUS\d+", query)                   # pattern match on all FUS-prefixed options
    return match.group(0) if match else None

def check_relevance(doc, user_option):
    return user_option in doc.page_content

In [ ]:
def chatbot():
    # Activate chatbot
    while True:
        user_message = input("User: ")
        if user_message.lower() == "quit":
            break

        # Calculate relevance scores and pick the most suitable chunk for response  - PREVIOUS APPROACH
        relevance_scores = []

        user_option = extract_details(user_message)  

        for index, doc in enumerate(docs):
            if user_option and user_option in doc.page_content:
                relevance_score = 10  
            else:
                relevance_score = 0  

            relevance_scores.append(relevance_score)

        most_relevant_chunk_index = relevance_scores.index(max(relevance_scores))
        most_relevant_chunk = docs[most_relevant_chunk_index]

        print(most_relevant_chunk)

        # Prompt using Chain-of-Thought technique

        metadata_info = (
                f"The document is structured in a tabular format with columns such as {', '.join(doc.metadata['columns'])}. "
                f"Each row in the table represents a unique dataset, where entries are arranged following a similar structure: (row number)| Option | Name | AFI | APA | BNL | CEE | DAC | FRA | GRC | IBE | IIG | ISC | JPN | LAT | MET | NAM | NOR | RCA | UKI | Sum. "
                f"For example, one entry might be formatted like this: '| 4 | FUS6140 | CIVCO eTRAX Sensor 12G | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 2 |'. "
                f"This example shows how each column is filled with specific data points relevant to each dataset. "
                f"Note that the actual content will be structured similarly, ensuring that each entry is clearly delineated by these columns."
            )
        
        # Specific examples in the prompt are key to successful execution. Tabular data requires a lot of handholding!
        response_prompt = (f"To address the query '{user_message}', examine the document which follows this metadata: {metadata_info}." 
                           f"Identify which of the columns are most likely to contain relevant information. Analyze the data, focusing on these columns, " 
                           f"to locate and extract the precise information needed. Extract required information from this section: \"{most_relevant_chunk.page_content}\."
                           f"Your response should follow this example: For option FUS6132 APA value is 2. If requested all values for an option, give only all columns that do not equal 0."
                           f"Your response should clearly indicate which columns are relevant and their corresponding values, similar to this example: "
                           f"'For option FUS9170, the relevant columns and their values are APA: 13, CEE: 2, DAC: 2, GRC: 27, IIG: 1, ISC: 1, LAT: 1, MET: 7, NAM: 319, RCA: 1, Sum: 374.'")

        # Must redo entire structure of the chatbot if want to integrate summary statistics due to parsing method. 

        chat_response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",
            prompt=response_prompt,
            max_tokens=150
        )

        chat_message = chat_response['choices'][0]['text'].strip()
        print(f"Bot: {chat_message}")

        xlsx_log("chat_log", user_message, chat_message)

        # Modify as needed to make sure API calls don't exceed the limit
        time.sleep(3)


In [ ]:
chatbot()  # Note that outputs tend to get more precise later in the session